In [ ]:
using LinearAlgebra
using Statistics
using Random

function generate_data()
    Random.seed!(123)
    n = 800
    
    area = rand(30:100, n)
    rooms = rand(1:3, n)
    age = rand(1:25, n)
    floor = rand(1:5, n)
    location = rand(1.0:20.0, n)
    
    X = hcat(ones(n), area, rooms, age, floor, location)
    
    coeffs = [2000000, 80000, 500000, -30000, 40000, -60000]
    
    y = X * coeffs + randn(n) * 400000
    
    return X, y, ["Константа", "Площадь", "Комнаты", "Возраст", "Этаж", "Расположение"]
end

function split_data(X, y, test_ratio=0.25)
    n = size(X, 1)
    idx = shuffle(1:n)
    test_n = Int(round(n * test_ratio))
    
    train_idx = idx[1:end-test_n]
    test_idx = idx[end-test_n+1:end]
    
    return X[train_idx, :], y[train_idx], X[test_idx, :], y[test_idx]
end

function fit_linear_regression(X, y)
    return X \ y
end

function calculate_metrics(X_test, y_test, coeffs)
    y_pred = X_test * coeffs
    
    mse = mean((y_pred - y_test).^2)
    rmse = sqrt(mse)
    mae = mean(abs.(y_pred - y_test))
    r2 = 1 - sum((y_test - y_pred).^2) / sum((y_test .- mean(y_test)).^2)
    
    println("Метрики модели:")
    println("MSE: ", round(mse, digits=2))
    println("RMSE: ", round(rmse, digits=2), " руб")
    println("MAE: ", round(mae, digits=2), " руб")
    println("R²: ", round(r2, digits=4))
    
    return y_pred, (mse, rmse, mae, r2)
end

function main()
    println("Модель прогнозирования цен на недвижимость")
    println("="^45)
    
    X, y, names = generate_data()
    X_train, y_train, X_test, y_test = split_data(X, y)
    
    coeffs = fit_linear_regression(X_train, y_train)
    
    println("\nКоэффициенты модели:")
    for i in 1:length(names)
        println(names[i], ": ", round(coeffs[i], digits=2))
    end
    
    predictions, metrics = calculate_metrics(X_test, y_test, coeffs)
    
    println("\nПример прогнозирования:")
    new_apt = [1, 65, 2, 10, 3, 5.0]
    price = dot(new_apt, coeffs)
    
    println("Площадь: 65 м²")
    println("Комнаты: 2")
    println("Возраст: 10 лет")
    println("Этаж: 3")
    println("Расположение: 5 км от центра")
    println("Предсказанная цена: ", round(price, digits=2), " руб")
    
    return coeffs, metrics
end

main()

# Комментарии по коду

## Функция generate_data
Функция генерирует данные из 800 наблюдений для обучения модели.

Характеристики данных:
- Площадь квартиры
- Количество комнат  
- Возраст дома
- Этаж квартиры
- Расположение (расстояние до центра)

Эти характеристики заносятся в матрицу X.

Также есть истинные коэффициенты квартиры:
- Базовая стоимость квартиры: 2.000.000 рублей
- Площадь квартиры (+80.000 рублей / м²)
- Количество комнат (+500.000 рублей / комната)
- Возраст дома (-30.000 рублей / год)
- Этаж квартиры (+40.000 рублей / этаж)
- Расположение (-60.000 рублей / км от центра)

В итоге в переменной y мы получаем вектор цен для 800 квартир

## Функция split_data
Функция разделяет данные на обучающую (75%) и тестовую (25%) выборки. Для того, чтобы не было упорядоченности данных массив из 800 чисел перемешивается.

## Функция fit_linear_regression
Функция находит коэффициенты линейной модели, которые лучше всего описывают зависимость между характеристиками и ценой.

## Функция calculate_metrics
Функция оценивает точность модели, используя различные метрики.

Используемые метрики:
- MSE (Среднеквадратичное отклонение)
- RMSE (Среднеквадратическая ошибка)
- MAE (Средняя абсолютная ошибка)
- R² (Коэффициент детерминации)

## Функция main
Основная функция программы, в которой выводится пример прогнозирования цены для новой квартиры.